In [18]:
import pandas as pd
import glob
import os
import cv2
import shutil
import csv

cwd = os.getcwd()
'''
file = os.path.join(cwd + "\data", "*.csv")
file = glob.glob(file)
df = pd.concat(map(pd.read_csv, file), ignore_index=False)
'''
df = pd.read_csv(cwd + "\data\legend.csv")
df.to_csv("all_data.csv")

In [19]:
df

,user_id,image,emotion
0,628,facial-expressions_2868588k.jpg,anger
1,628,facial-expressions_2868585k.jpg,surprise
2,628,facial-expressions_2868584k.jpg,disgust
3,628,facial-expressions_2868582k.jpg,fear
4,dwdii,Aaron_Eckhart_0001.jpg,neutral
...,...,...,...
13685,jhamski,SharmilaTagore_80.jpg,HAPPINESS
13686,jhamski,SharmilaTagore_81.jpg,HAPPINESS
13687,jhamski,SharmilaTagore_82.jpg,HAPPINESS
13688,jhamski,SharmilaTagore_83.jpg,HAPPINESS


In [22]:
def variance_of_laplacian(image):
	return cv2.Laplacian(image, cv2.CV_64F).var()

cwd = os.getcwd()

dir_img = cwd + '/cleaned_images'
if os.path.exists(dir_img):
    shutil.rmtree(dir_img)
os.makedirs(dir_img)

all_image = []
count = 0

for idx, row in df.iterrows():
    
    imagePath = cwd + "/images/" + row.image

    image = cv2.imread(imagePath)
    height, width, channels = image.shape

    # check size image
    if(width != 350 or height != 350):
        continue

    # change color
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # check blurry
    fm = variance_of_laplacian(gray)
    if fm < 10:
        continue
    
    # detect face
    face_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_frontalface_alt.xml')
    eye_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_eye_tree_eyeglasses.xml')
    smile_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_smile.xml')

    face = face_cascade.detectMultiScale(
        gray,
        scaleFactor = 1.1,
        minNeighbors = 4,
        minSize = (200, 200),
        flags = cv2.CASCADE_SCALE_IMAGE
    )
    
    for (x, y, w, h) in face:
        roi_gray = gray[y:y+h, x:x+w]

    smile = smile_cascade.detectMultiScale(
        roi_gray,
        scaleFactor = 1.16,
        minNeighbors = 35,
        minSize = (25, 25),
        flags = cv2.CASCADE_SCALE_IMAGE
    )

    eyes = eye_cascade.detectMultiScale(roi_gray)

    if len(face) != 1 or len(smile) < 1 or len(eyes) < 2:
        continue

    all_image.append({row.user_id, row.image, row.emotion})
    cv2.imwrite(os.path.join(dir_img, row.image), gray)
    count += 1
print(count)

{'Abdel_Nasser_Assidi_0002.jpg', '302', 'neutral'}
0


In [21]:
header = ['user.id', 'image', 'emotion']

f = open(cwd + '/preprocessing_data.csv', 'w')
writer = csv.writer(f)
writer.writerow(header)
writer.writerow(all_image)
f.close()